In [11]:
# multi-class classification with Keras
#libraries

# multi-class classification with Keras
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.models import model_from_json
import sys 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import matplotlib.pyplot as plt # this is used for the plot the graph 
import pickle
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import PowerTransformer
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
import numpy as np # linear algebra
from sklearn.model_selection import train_test_split



In [2]:
def NaiveBayes(data):

    testx = normalise_data(data)
    
    filenamenb = 'naive_bayes_final_model.sav'
    loaded= pickle.load(open(filenamenb,'rb'))
    droppedtest=testx.drop(columns=['Temperature (C)'])##,'Apparent Temperature (C)'])
    pred=loaded.predict(droppedtest)
    
    labels={
    1: 'Partly Cloudy',
    2: 'Mostly Cloudy',
    3: 'Overcast',
    4: 'Clear',
    5: 'Foggy',
    }
    
    lableprint=pd.DataFrame(data=pred,columns=["naive_bayes"])
    lableprint["naive_bayes"]=lableprint["naive_bayes"].map(labels)
    return lableprint

In [3]:
def KNN(data):
#     testx
    testx = normalise_data(data)
    filenameknn = 'knn_final_model.sav'
    loaded= pickle.load(open(filenameknn,'rb'))
    pred=loaded.predict(testx)
    
    labels={
    1: 'Partly Cloudy',
    2: 'Mostly Cloudy',
    3: 'Overcast',
    4: 'Clear',
    5: 'Foggy',
    }
    
    lableprint=pd.DataFrame(data=pred,columns=["knn"])
    lableprint["knn"]=lableprint["knn"].map(labels)
    return lableprint

In [4]:
def DecisionTree(data):
#     TestX
    treedata = np.array(data["Visibility (km)"])
    treedata = treedata.reshape(-1,1)
    TestX = treedata;
    #Load the model
    filename = 'DecisionTreeClassifier.sav'
    selectedPredictors = ['Temperature (C)', 'Apparent Temperature (C)', 'Humidity', 'Wind Bearing (degrees)', 'Visibility (km)', 'Pressure (millibars)']
    mod = pickle.load(open(filename, 'rb')) #This is the model that was best
    
    #Predict the output for the given data
    predicted = mod.predict(TestX) #Predict stuff
    
    #Convert back to the label types
    label_map={
        1: 'Partly Cloudy',
        2: 'Mostly Cloudy',
        3: 'Overcast',
        4: 'Clear',
        5: 'Foggy',
    }
    
    #Put the labels back in replace of their numeric equivilents
    df = pd.DataFrame(data=predicted,  columns=["decision_tree"])
    df['decision_tree'] = df["decision_tree"].map(label_map)
    return df

In [5]:
def RandomForest(data):
#     TestX
#     TestX = data.drop(['Temperature (C)','Apparent Temperature (C)','Wind Bearing (degrees)'],axis=1)
    TestX = data
    filename = 'RandomForestClassifier.sav'
    selectedPredictors = ['Humidity', 'Wind Speed (km/h)', 'Visibility (km)', 'Pressure (millibars)']
    mod = pickle.load(open(filename, 'rb')) #This is the model that was best
    
    predicted = mod.predict(TestX) #Predict stuff
    
    #Convert back to the label types
    label_map = {
        4:'Partly Cloudy',
        2:'Mostly Cloudy',
        3:'Overcast',
        0:'Clear',
        1:'Foggy'
    }
    
    df = pd.DataFrame(data=predicted,  columns=["random_forest"])
    df['random_forest'] = df["random_forest"].map(label_map)
    return df

In [12]:
def ANN(data):

    X = normalise_data(data);
    tX = np.array(X);
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    lmodel = model_from_json(loaded_model_json)
    lmodel.load_weights("Annmodel.h5")
    lmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    results = lmodel.predict(tX)
    predresults = tf.keras.backend.argmax(results).numpy()
    data = pd.DataFrame(data=predresults, columns=["ann"])
    
    label_map = {
    0:'Partly Cloudy',
    1:'Mostly Cloudy',
    2:'Overcast',
    3:'Clear',
    4:'Foggy',
    }
    data['ann'] = data['ann'].map(label_map)
    return data

In [7]:
def load_dataset(path):
    data = pd.read_csv(path)
    data = data.drop(['Unnamed: 0'],axis=1);
    return data

In [8]:
def normalise_data(data):
    ndata = load_dataset("trainx.csv")
    data = (data - ndata.min())/(ndata.max()-ndata.min());
    return data

In [9]:
def pred_results(data):
    mtree = DecisionTree(data)
    mforest = RandomForest(data)
    mnb = NaiveBayes(data)
    mknn = KNN(data)
    mann = ANN(data)

    results = mtree
    results['random_forest'] = mforest['random_forest']
    results['knn'] = mknn['knn']
    results['naive_bayes'] = mnb['naive_bayes']
    results['ann'] = mann['ann']
    
    testy = load_dataset("testy.csv")
    label_map = {
    1:'Partly Cloudy',
    2:'Mostly Cloudy',
    3:'Overcast',
    4:'Clear',
    5:'Foggy',
    }
    label_map_inv = {
    'Partly Cloudy':1,
    'Mostly Cloudy':2,
    'Overcast':3,
    'Clear':4,
    'Foggy':5,
    }
    testy['summary'] = testy['summary'].map(label_map)
    # testing accuracies 
    cols = results.columns
    accuracies = list()
    count = 0;
    for i in range(len(cols)):
        comp = np.where(results[cols[i]] == testy["summary"], True, False)
        values, counts = np.unique(comp, return_counts=True)
        accuracies.append(counts[1]*100/len(results))
        print(cols[i]+" : " + str(counts[1]*100/len(results)))

In [13]:
data = load_dataset("testx.csv")
pred_results(data)

C:\Users\yashb\Anaconda3\envs\nlp_project\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator GaussianNB from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\yashb\Anaconda3\envs\nlp_project\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


decision_tree : 47.77821886992193
random_forest : 23.355829032684927
knn : 53.14807463279079
naive_bayes : 47.227735874024084
ann : 50.0992457324335
